In [3]:
using FermiCG
using Printf
using Test
using JLD2 

@load "../test/_testdata_cmf_h12_64bit.jld2"

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, d1.a, d1.b);

v_bst = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases)
v_tps = FermiCG.TPSCIstate(clusters, FockConfig(init_fspace))
FermiCG.eye!(v_tps)

s_bst = FermiCG.build_compressed_1st_order_state(v_bst, cluster_ops, clustered_ham, nbody=4, thresh=1e-24)
s_tps = FermiCG.open_matvec_thread(v_tps, cluster_ops, clustered_ham, thresh=1e-32)

@show size(s_bst);
@show size(s_tps);





 Remove duplicates
 Number of terms reduced from  2525 to  1105
IDX001:DIM0016:001|002|


IDX002:DIM0016:003|004|


IDX003:DIM0256:005|006|007|008|


IDX004:DIM0016:009|010|


IDX005:DIM0016:011|012|


...

 In open_matvec_thread
 Setup threaded jobs:                                0.000178 seconds (1.58 k allocations: 145.469 KiB)
 Number of jobs:    525
 Number of threads: 4


 Compute matrix-vector:                            

  0.006038 seconds (78.96 k allocations: 6.596 MiB)


 Now collect thread results :                      

  0.000148 seconds (1.06 k allocations: 64.359 KiB)
size(s_bst) = (16306, 1)
size(s_tps) = (49378, 1)


In [4]:
e_tpsci, v_tpsci = FermiCG.tpsci_ci(v_tps, cluster_ops, clustered_ham, thresh_cipsi=1e-4, thresh_foi=1e-6, thresh_asci=-1)
e_pt = FermiCG.compute_pt2_energy(v_tpsci, cluster_ops, clustered_ham, thresh_foi=1e-8, verbose=0);

 ci_vector     : (1, 1)
 thresh_cipsi  : 0.0001
 thresh_foi    : 1.0e-6
 thresh_asci   : -1
 thresh_var    : nothing
 max_iter      : 10
 conv_thresh   : 0.0001
 nbody         : 4
 incremental   : true
 ci_conv       : 1.0e-5
 ci_max_iter   : 50
 ci_max_ss_vecs: 12
 davidson      : false
 max_mem_ci    : 20.0
 threaded      : true




     Selected CI Iteration:    1 epsilon:   0.00010000
 Memory needed to hold full CI matrix:   0.00000001 (Gb) Max allowed:  20.00000000 (Gb)



 |== Tensor Product State CI =======================================
 Hamiltonian matrix dimension =     1: 


 Build full Hamiltonian matrix with dimension:     

  0.260406 seconds (709.54 k allocations: 37.108 MiB, 99.67% compilation time)
 Now diagonalize


...

 Compute <0|H|0>:                                  

  1.493467 seconds (80.17 M allocations: 2.279 GiB, 29.45% gc time)


 Number of jobs:    4571
 Number of threads: 4


 Time spent computing E2          6.9 (s)


  Root         E(0)         E(2)
     1 -18.33015303 -18.33020256
 ......................................................................................|


# BST PT2

In [5]:
e_var = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);
e_pt = FermiCG.compute_pt2_energy(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-12, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                

  2.002725 seconds (3.06 M allocations: 183.022 MiB, 99.40% compilation time)


 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 


 Diagonalization time:                               0.790178 seconds
 Compute <S^2>:                                    

  0.003450 seconds (2.06 k allocations: 443.771 KiB, 81.73% compilation time)


...

-

-

-

-------------|


 Time spent computing E2:                              11.3 s Allocated:    1.5e+00 GB
 State   1: E(PT2) corr:                          -0.00992615
  Root         E(0)         E(2)
     1 -18.31710895 -18.32703510
 ......................................................................................|


In [6]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-33)
    
# FermiCG.project_out!(s_bst, v, thresh=1e-32)
# @show size(s_bst)
# s_bst = FermiCG.compress(s_bst, thresh=1e-32)
# @show size(s_bst)
@time v_pt, e_pt = FermiCG.hylleraas_compressed_mp2(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                

  0.010290 seconds (4.08 k allocations: 30.747 MiB)


 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 


 Diagonalization time:                               0.001084 seconds
 Compute <S^2>:                                    

  0.000466 seconds (1.35 k allocations: 399.688 KiB)


...

 16	8.33e-08


 17	3.61e-08


 18	1.74e-08


 19	8.26e-09

 Time to solve for PT1 with conjugate gradient:      2.427479 seconds   1.18e+00 Gb


 Compute <0|H|1>:                                    0.339014 seconds   2.91e-02 Gb
 SxC[r]  -0.00000000 SxSx   1.00000000
 State   1: E(PT2) corr:                          -0.00992622
 State   1: E(PT2):                              -18.32703517
  3.780144 seconds (18.33 M allocations: 1.316 GiB, 7.89% gc time, 77.04% compilation time)


In [7]:
@time e_pt, v_pt = FermiCG.compute_pt1_wavefunction(s_bst, v, cluster_ops, clustered_ham);


 |...................................BST-PT2............................................


 Length of input      FOIS:                             16406


 Compute <X|H|0>:                                    0.019779 seconds   1.10e-02 Gb


 Compute <X|F|0>:                                    0.000746 seconds   9.81e-04 Gb
 State   1: E(PT2) corr:                          -0.00992622
  Root         E(0)         E(2)
     1 -18.31710895 -18.32703517
 ......................................................................................|
  0.107132 seconds (368.45 k allocations: 46.758 MiB, 35.03% compilation time)


# Pseudo-Canonicalization

In [8]:
r_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

v = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-3)

e_var, v = FermiCG.ci_solve(v, cluster_ops, clustered_ham); 

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-4)

# @time v_pt, e_pt = FermiCG.hylleraas_compressed_mp2(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                

  0.004971 seconds (4.08 k allocations: 30.747 MiB)


 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 


 Diagonalization time:                               0.001095 seconds
 Compute <S^2>:                                    

  0.000432 seconds (1.35 k allocations: 399.688 KiB)


...

 Diagonalization time:                               2.739043 seconds
 Compute <S^2>:                                    

  0.038323 seconds (389.26 k allocations: 26.057 MiB)


  Root       Energy           S2
     1 -18.32994631   0.00014089
 ==================================================================|
 Number of tasks:                                         661
 Compute tasks:                                    


 Time building compressed vector:                    2.551300 seconds   2.25e+00 Gb GC: 6.3e-01 sec


 Add results together:                               0.653650 seconds   1.97e-01 Gb GC: 0.0e+00 sec


In [9]:
@time v_pt, e_pt = FermiCG.compute_pt1_wavefunction(s_bst, v, cluster_ops, clustered_ham, verbose=1);


 |...................................BST-PT2............................................


 Length of input      FOIS:                             30373


 Compute <X|H|0>:                                    2.317532 seconds   1.74e+00 Gb


 Compute <X|F|0>:                                    0.014122 seconds   1.36e-02 Gb
 State   1: E(PT2) corr:                          -0.00021217
  Root         E(0)         E(2)
     1 -18.32994631 -18.33015848
 ......................................................................................|
  3.196717 seconds (26.24 M allocations: 2.147 GiB, 11.83% gc time, 2.90% compilation time)


In [10]:
@time v_pt, e_pt = FermiCG.compute_pt1_wavefunction(v, cluster_ops, clustered_ham, verbose=1, thresh_foi=1e-4);

 Number of tasks:                                         661
 Compute tasks:                                    


 Time building compressed vector:                    2.619656 seconds   2.24e+00 Gb GC: 7.8e-01 sec


 Add results together:                               0.132165 seconds   9.40e-02 Gb GC: 0.0e+00 sec


 Compute Compressed FOIS:                            2.752843 seconds   2.34e+00 Gb

 |...................................BST-PT2............................................


 Length of input      FOIS:                             30373


 Compute <X|H|0>:                                    2.228038 seconds   1.74e+00 Gb


 Compute <X|F|0>:                                    0.014392 seconds   1.36e-02 Gb
 State   1: E(PT2) corr:                          -0.00021217
  Root         E(0)         E(2)
     1 -18.32994631 -18.33015848
 ......................................................................................|
  5.780186 seconds (51.57 M allocations: 4.319 GiB, 18.00% gc time, 0.38% compilation time)


In [14]:
@time e_pt = FermiCG.compute_pt2_energy(v, cluster_ops, clustered_ham, verbose=0, thresh_foi=1e-8);


 |...................................BST-PT2............................................
 Solve zeroth-order problem:                       
 |== BST CI ========================================================
 Solve CI with # variables:                               651
 Cache Hamiltonian:                                

  0.463890 seconds (4.34 M allocations: 332.750 MiB)


 Iter:   1 SS: 1    E:  -18.32994631   R: 7.9e-06   LinDep: 2.2e-16* 


 Iter:   2 SS: 2    E:  -18.32994631   R: 3.6e-06   LinDep: 0.0e+00* 


 Iter:   3 SS: 3    E:  -18.32994631   R: 1.6e-06   LinDep: 0.0e+00* 


...

  0.040566 seconds (390.17 k allocations: 26.113 MiB)


  Root       Energy           S2
     1 -18.32994631   0.00014095
 ==================================================================|
 Diagonalization time:                               1.355918 seconds
 Compute <0|H0|0>:                                 

  0.008907 seconds (83.81 k allocations: 6.452 MiB)


 Number of jobs:    4009
 Number of threads: 4


 Time spent computing E2:                              16.4 s Allocated:    1.9e+01 GB
 State   1: E(PT2) corr:                          -0.00021493
  Root         E(0)         E(2)
     1 -18.32994631 -18.33016125
 ......................................................................................|
 18.066178 seconds (147.65 M allocations: 18.629 GiB, 17.82% gc time, 1.44% compilation time)


In [17]:
ebst, vbst = FermiCG.block_sparse_tucker(v, cluster_ops, clustered_ham, thresh_var=1e-2, thresh_foi=1e-6, thresh_pt=1e-3)

 max_iter         : 20
 max_iter_pt      : 200
 nbody            : 4
 H0               : Hcmf
 thresh_var       : 0.01
 thresh_foi       : 1.0e-6
 thresh_pt        : 0.001
 ci_conv          : 1.0e-5
 ci_max_iter      : 50
 ci_max_ss_vecs   : 12
 ci_lindep_thresh : 1.0e-10
 resolve_ss       : false
 do_pt            : true
 tol_tucker       : 1.0e-6



     BST Iteration:    1 epsilon:   0.01000000
 Ref state compressed from:                               651 → 41         (thresh =  1.0e-02)
 Compute zeroth-order energy:                      

  0.052424 seconds (457.23 k allocations: 33.701 MiB)
 Compute <S^2>:                                    

  0.006188 seconds (49.77 k allocations: 3.550 MiB)
  Root       Energy           S2
     1 -18.32639340   0.00001933



 Compute FOIS. Reference space dim:                        41
 Number of tasks:                                         661
 Compute tasks:                                    


 Time building compressed vector:                    1.322430 seconds   1.28e+00 Gb GC: 0.0e+00 sec


...

 Iter:   2 SS: 2    E:  -18.32906391   R: 8.8e-03   LinDep: 2.2e-16* 


 Iter:   3 SS: 3    E:  -18.32911288   R: 2.3e-03   LinDep: 6.7e-16* 


 Iter:   4 SS: 4    E:  -18.32911638   R: 8.8e-04   LinDep: 7.8e-16* 


 Iter:   5 SS: 5    E:  -18.32911692   R: 2.5e-04   LinDep: 2.4e-15* 


 Iter:   6 SS: 6    E:  -18.32911699   R: 1.2e-04   LinDep: 1.1e-16* 


In [ ]:
ebst, vbst = FermiCG.block_sparse_tucker_new(v, cluster_ops, clustered_ham, thresh_var=1e-2, thresh_foi=1e-6, thresh_pt=1e-3)